<a href="https://colab.research.google.com/github/Bhabuk10/movie_recommendation_system/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Data collection and Preprocessing

In [2]:
movies= pd.read_csv("/content/drive/MyDrive/movies.csv")

In [3]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [5]:
movies.shape

(4803, 24)

In [6]:
# selecting relevant columns for feature_extraction
selected_features= ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
# replacing null/missing values with null string 

for feature in selected_features:
  movies[feature]= movies[feature].fillna(' ')

In [8]:
# combining all the 5 selected features 

combined_features= movies['genres']+movies['keywords']+movies['tagline']+movies['cast']+movies['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy Romance A newlywed couple's honeymoon i...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801      A New Yorker in ShanghaiDaniel Henney Eliza ...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [9]:
combined_features.shape

(4803,)

In [10]:
# converting textual data to feature vectors/numerical values

vectorizer= TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 3519)	0.16965125707312265
  (0, 20128)	0.27423341520002953
  (0, 16079)	0.15504654185850117
  (0, 14027)	0.22398765118158517
  (0, 22585)	0.1606699814283307
  (0, 25788)	0.19665600002389527
  (0, 21804)	0.2023479784763536
  (0, 20833)	0.21579678353924853
  (0, 26765)	0.19538155986837238
  (0, 26410)	0.23512284622451368
  (0, 20866)	0.15574033593282205
  (0, 17987)	0.2615750181395899
  (0, 17593)	0.08702748804687763
  (0, 26389)	0.12637191166265954
  (0, 23496)	0.0749175019250534
  (0, 22112)	0.27423341520002953
  (0, 4594)	0.2456273238868167
  (0, 25580)	0.1301518979110662
  (0, 22244)	0.3371492353507311
  (0, 9739)	0.16507437746091785
  (0, 4378)	0.2182956727291268
  (0, 8929)	0.27423341520002953
  (0, 21181)	0.09980259346191937
  (0, 8571)	0.11770025292082674
  (0, 429)	0.09108551782969122
  :	:
  (4801, 21507)	0.31553026282738933
  (4801, 26587)	0.31553026282738933
  (4801, 26738)	0.3009656360165369
  (4801, 7793)	0.2577179823103029
  (4801, 727)	0.19276282194800867
  (4801, 1

In [12]:
# getting similarity score / similarity confidence value using cosine_similarity

similarity= cosine_similarity(feature_vectors)
print(similarity)


[[1.         0.0685423  0.01490125 ... 0.         0.         0.        ]
 [0.0685423  1.         0.02801055 ... 0.01232616 0.         0.        ]
 [0.01490125 0.02801055 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01232616 0.         ... 1.         0.         0.02861645]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02861645 0.         1.        ]]


In [13]:
similarity.shape

(4803, 4803)

In [14]:
# movie name given by user

movie_name= input("Enter your favourite movie name: ")

Enter your favourite movie name: Avatar


In [15]:
# creating a list of all the movies names in the dataset

list_of_titles = movies['title'].tolist()
print(list_of_titles )

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [16]:
# finding close match for movie name given by the user

find_close_match= difflib.get_close_matches (movie_name , list_of_titles)
print(find_close_match)

['Avatar']


In [17]:
close_match= find_close_match[0]


In [18]:
# finding the index of the movie with the title

index = movies[movies.title == close_match]['index'].values[0]
print(index)
type(index)

0


numpy.int64

In [19]:
index1 = movies[movies.title == close_match]['index'].values
print(index1)
type(index1)

[0]


numpy.ndarray

In [20]:
similarity_score= list(enumerate(similarity[index]))
print(similarity_score)

[(0, 0.9999999999999998), (1, 0.0685423021024959), (2, 0.014901248384876182), (3, 0.006747502754466987), (4, 0.09525190640613791), (5, 0.028313351447333917), (6, 0.008534629384098468), (7, 0.024584963078151504), (8, 0.03133892006229902), (9, 0.014572841326611968), (10, 0.0689796540023492), (11, 0.013433190961298606), (12, 0.022596711581231246), (13, 0.019467953156918353), (14, 0.060897068192689426), (15, 0.008033707105993175), (16, 0.015441654003938862), (17, 0.014202917754522384), (18, 0.021136266411947206), (19, 0.029139412686312045), (20, 0.021288337563183844), (21, 0.005490015288398053), (22, 0.020546915581294026), (23, 0.03776094090373469), (24, 0.03529071103720568), (25, 0.029400541567390748), (26, 0.06236444091928445), (27, 0.02207477340838443), (28, 0.023784915296585038), (29, 0.0153199453763075), (30, 0.029595155399899337), (31, 0.052245136954888534), (32, 0.02557397051597167), (33, 0.02476252482197287), (34, 0.0), (35, 0.01724221091990328), (36, 0.03249165798287092), (37, 0.0

In [21]:
len(similarity_score)

4803

In [22]:
# sorting movies based on their similarity score

sorted_similar_movies= sorted(similarity_score , key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(0, 0.9999999999999998), (3158, 0.23764435280351745), (94, 0.20499805144136318), (1531, 0.16182771112298275), (1951, 0.15497768609119145), (838, 0.15236607999399127), (2403, 0.1478100121166943), (206, 0.14702839992698155), (1759, 0.14082604314024388), (56, 0.1329750493954333), (1473, 0.12527841813863455), (1053, 0.12230881490608068), (1650, 0.11989228241216901), (3730, 0.1191179756658204), (1099, 0.11906096535439695), (47, 0.11838572789046864), (43, 0.11800100720345276), (278, 0.11719351419214614), (3105, 0.11584065431883056), (4108, 0.11467034723959739), (1275, 0.11360140808874838), (812, 0.11170475088371923), (942, 0.11087987142707918), (2060, 0.10832550435870314), (1804, 0.10632897993008253), (3056, 0.10375412753788381), (977, 0.10362175419657464), (305, 0.10249110221986969), (158, 0.10214060149074296), (3208, 0.10170314733277294), (150, 0.101622295692894), (2827, 0.10019841033503869), (3439, 0.09881627572649743), (342, 0.09846075131044228), (775, 0.09730673753423724), (1922, 0.095

In [23]:
# print name of similar movies based on the index 

print("movies suggested for users : \n")

i=1 
for movie in sorted_similar_movies:
  index= movie[0]
  title_from_index= movies[movies.index== index]['title'].values[0]
  
  if(i<=10):
    print(i,'.', title_from_index)
    i+=1


movies suggested for users : 

1 . Avatar
2 . Alien
3 . Guardians of the Galaxy
4 . Moonraker
5 . Space Dogs
6 . Alien³
7 . Aliens
8 . Clash of the Titans
9 . The Right Stuff
10 . Star Trek Beyond


Movie_recommendation_system

In [25]:
# combing all above codes to finally build a recommendation system

movie_name= input("Enter your favourite movie name: ")

list_of_titles = movies['title'].tolist()

find_close_match= difflib.get_close_matches (movie_name , list_of_titles)

close_match= find_close_match[0]

index = movies[movies.title == close_match]['index'].values[0]

similarity_score= list(enumerate(similarity[index]))

sorted_similar_movies= sorted(similarity_score , key = lambda x:x[1], reverse = True)

print("movies suggested for users : \n")

i=1 
for movie in sorted_similar_movies:
  index= movie[0]
  title_from_index= movies[movies.index== index]['title'].values[0]
  
  if(i<=10):
    print(i,'.', title_from_index)
    i+=1


Enter your favourite movie name: The dark night rises
movies suggested for users : 

1 . The Dark Knight Rises
2 . The Dark Knight
3 . Batman Begins
4 . The Prestige
5 . Interstellar
6 . Batman & Robin
7 . Get Smart
8 . London Has Fallen
9 . Superman
10 . Child 44
